In [11]:
import numpy
import re
import pandas as pd
import numpy as np
import keras
import string
import nltk

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

string.punctuation = string.punctuation +'“'+'”'+'-'+'’'+'‘'+'—'
string.punctuation = string.punctuation.replace('.', '')

In [14]:
import nltk
nltk.download('punkt')
raw_text = open('/content/Sherlock.txt', encoding = 'utf8').read()

file_nl_removed = ""
for line in raw_text:
  line_nl_removed = line.replace("\n", " ")           
#removes newlines
  file_nl_removed += line_nl_removed

file_p = "".join([char for char in file_nl_removed if char not in string.punctuation])   
#removes all special characters
sents = nltk.sent_tokenize(file_p)
print("The number of sentences is", len(sents)) 
#prints the number of sentences

string.punctuation = string.punctuation + '.'
file_q = "".join([char for char in file_p if char not in string.punctuation])   #removes even periods.
words = nltk.word_tokenize(file_q)
print("The number of tokens is", len(words)) 
#prints the number of tokens

average_tokens = round(len(words)/len(sents))
print("The average number of tokens per sentence is", average_tokens) 
#prints the average number of tokens per sentence

unique_tokens = set(words)
print("The number of unique tokens are", len(unique_tokens)) 
#prints the number of unique tokens

preprocessed_text = file_p.lower()       
#converts corpus into lowercase

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


The number of sentences is 5844
The number of tokens is 107598
The average number of tokens per sentence is 18
The number of unique tokens are 9414


In [15]:
# Hyperparameters of the model
vocab_size = 2750  #chosen based on statistics of the model
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'
# tokenizes sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([preprocessed_text])
word_index = tokenizer.word_index
seq_length = 50
tokens = tokenizer.texts_to_sequences([preprocessed_text])[0]

In [16]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size is -", N)
X = numpy.array(dataX)

# one hot encodes the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

Total training data size is - 99319


In [17]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])

# compiles model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 100)           275000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 2750)              354750    
                                                                 
Total params: 714,230
Trainable params: 714,230
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Uses validation split of 0.2 while training
num_epochs = 5
history = model.fit(X, y, epochs=num_epochs, batch_size = 128, verbose=1, validation_split=0.2)

Epoch 1/5
621/621 [==============================] - 168s 262ms/step - loss: 5.9601 - accuracy: 0.0635 - val_loss: 5.9918 - val_accuracy: 0.0680
Epoch 2/5
621/621 [==============================] - 157s 252ms/step - loss: 5.5435 - accuracy: 0.0916 - val_loss: 5.7513 - val_accuracy: 0.0936
Epoch 3/5
621/621 [==============================] - 157s 253ms/step - loss: 5.3031 - accuracy: 0.1104 - val_loss: 5.6362 - val_accuracy: 0.1079
Epoch 4/5
621/621 [==============================] - 148s 238ms/step - loss: 5.1462 - accuracy: 0.1254 - val_loss: 5.5644 - val_accuracy: 0.1172
Epoch 5/5
621/621 [==============================] - 159s 256ms/step - loss: 5.0169 - accuracy: 0.1369 - val_loss: 5.5385 - val_accuracy: 0.1234


In [19]:
#Creates word to idx map using tokenizer.word_index
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
# Returns the next n words greedily
def next_tokens(input_str, n):
    print ("Seed -",  input_str, sep = '\n\n')
    final_string = ''
    for i in range(n):
        token = tokenizer.texts_to_sequences([input_str])[0]
        prediction = model.predict(token, verbose=0)
        final_string = final_string + reverse_word_map[numpy.argmax(prediction[0])] + ' ' 
        input_str = input_str + ' ' + reverse_word_map[numpy.argmax(prediction[0])]
        input_str = ' '.join(input_str.split(' ')[1:])
    return final_string

In [23]:
# picks a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ' '.join([reverse_word_map[value] for value in pattern])

output = next_tokens(input_str, 10)
print("\nGenerated string -\n\n", output)
#speficies an unseen input string
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Uses first 50 tokens from given input_str as input. Since the seq_length is 50, only 50 tokens are taken using the tokenizer.
output = next_tokens(input_str, 10)
print("\nGenerated string -\n\n", output)

Seed -

than when the facts slowly <OOV> before your own eyes and the mystery <OOV> <OOV> away as each new <OOV> <OOV> a step which <OOV> on to the complete truth at the time the circumstances made a deep impression upon me and the <OOV> of two years has hardly served


ValueError: ignored

In [25]:
# Uses the preprocessed data and create raw_text
raw_text = preprocessed_text   #periods have not been removed for better results

# creates mapping of unique characters to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# Prints the total characters and character vocab size
n_chars = len(raw_text)
n_vocab = len(chars)

print("The number of total characters are", n_chars)
print("\nThe character vocab size is", n_vocab)

The number of total characters are 564544

The character vocab size is 47


In [26]:
#Prepares dataset where the input is sequence of 100 characters and target is next character.
seq_length = 100

dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]

  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshapes X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# one hot encodes the output variable
y = np_utils.to_categorical(dataY)

Total Patterns:  564444


In [27]:
embedding_dim =100
max_length =100
model = Sequential()
model.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          4700      
                                                                 
 lstm_1 (LSTM)               (None, 256)               365568    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 46)                11822     
                                                                 
Total params: 382,090
Trainable params: 382,090
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs = 2, batch_size=300)

Exception ignored in: <function _xla_gc_callback at 0x7fa0dac7aee0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Epoch 1/2
  46/2823 [..............................] - ETA: 1:00:43 - loss: 1.5389